In [4]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
from sklearn import preprocessing

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
train_data = pd.read_csv('train/train_all.csv')
test_data = pd.read_csv('test/republish_test.csv')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def filln(data):
    data = data.replace('\N', -1)
    data.gender = data.gender.astype(int)
    data.age = data.age.astype(int)
    data['3_total_fee'] = data['3_total_fee'].astype(float)
    data['2_total_fee'] = data['2_total_fee'].astype(float)
    return data
    
def trans(data):
    data.current_service = data.current_service.replace(90063345, 0)
    data.current_service = data.current_service.replace(90109916, 1)
    data.current_service = data.current_service.replace(90155946, 2)
    data.current_service = data.current_service.replace(89950166, 3)
    data.current_service = data.current_service.replace(89950167, 4)
    data.current_service = data.current_service.replace(99999828, 5)
    data.current_service = data.current_service.replace(89950168, 6)
    data.current_service = data.current_service.replace(99999827, 7)
    data.current_service = data.current_service.replace(99999826, 8)
    data.current_service = data.current_service.replace(99999830, 9)
    data.current_service = data.current_service.replace(99999825, 10)
    return data

def trans_1(data):    
    data.service_type = data.service_type.replace(4,0)
    data.service_type = data.service_type.replace(3,0)
    
#     contract_type 3->2  9->3  12->4  7->5  2->7
    data.contract_type = data.contract_type.replace(2,10)
    data.contract_type = data.contract_type.replace(3,2)
    data.contract_type = data.contract_type.replace(9,3)
    data.contract_type = data.contract_type.replace(12,4)
    data.contract_type = data.contract_type.replace(7,5)
    data.contract_type = data.contract_type.replace(10,7)
    
    data.net_service = data.net_service.replace(9,1)
    
    data = data.loc[data.gender != -1]
    return data

In [7]:
def one_hot(data):
    data1 = data
    for i in range(0,2):
        data2 = data1.loc[data1.service_type == i,['user_id', 'service_type']]
        data2 = data2.groupby('user_id')['service_type'].size().reset_index().rename(columns = {'service_type':'stype_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')

    for i in range(0,2):
        data2 = data1.loc[data1.is_mix_service == i]
        data2 = data2.groupby('user_id')['is_mix_service'].size().reset_index().rename(columns = {'is_mix_service':'ismix_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')
        
    for i in range(0,2):
        data2 = data1.loc[data1.many_over_bill == i]
        data2 = data2.groupby('user_id')['many_over_bill'].size().reset_index().rename(columns = {'many_over_bill':'mob_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')
        
    for i in range(0,9):
        data2 = data1.loc[data1.contract_type == i]
        data2 = data2.groupby('user_id')['contract_type'].size().reset_index().rename(columns = {'contract_type':'ctype_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')
        
    for i in range(0,2):
        data2 = data1.loc[data1.is_promise_low_consume == i]
        data2 = data2.groupby('user_id')['is_promise_low_consume'].size().reset_index().rename(columns = {'is_promise_low_consume':'islow_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')
        
    for i in range(1,5):
        data2 = data1.loc[data1.net_service == i]
        data2 = data2.groupby('user_id')['net_service'].size().reset_index().rename(columns = {'net_service':'ns_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')    
    
    for i in range(0,3):
        data2 = data1.loc[data1.gender == i]
        data2 = data2.groupby('user_id')['gender'].size().reset_index().rename(columns = {'gender':'gender_' + str(i)})
        data = pd.merge(data,data2,on = 'user_id', how = 'left')
    return data

def predata(data):
    #最近几月总金额
    data['total_fee'] = data['1_total_fee'] + data['2_total_fee'] + data['3_total_fee'] + data['4_total_fee']
    data['1_2_total_fee'] = data['1_total_fee'] + data['2_total_fee']
    
    #连续出账月数
    data['fee_month'] = 0
    data.loc[data['1_total_fee'] >0]['fee_month'] += 1
    data.loc[data['2_total_fee'] >0]['fee_month'] += 1
    data.loc[data['3_total_fee'] >0]['fee_month'] += 1
    data.loc[data['4_total_fee'] >0]['fee_month'] += 1
    
    #总通话时长
    data['call_time'] = data['local_caller_time'] + data['service1_caller_time'] + data['service2_caller_time']
    
    #总流量
    data['traffic'] = data['last_month_traffic'] + data['local_trafffic_month']
    
    
    return data

# one_hot(train_data)

In [8]:
train_data = filln(train_data)
test_data = filln(test_data)
train_data = trans(train_data)
train_data = trans_1(train_data)
test_data = trans_1(test_data)

In [9]:
train_data = one_hot(train_data)
test_data = one_hot(test_data)

In [10]:
train_data = predata(train_data)
test_data = predata(test_data)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [11]:
# feats = ['service_type', 'is_mix_service', 'many_over_bill', 'contract_type', 'is_promise_low_consume',
#              'net_service', 'gender']
# for feat in feats:
#     print train_data[feat].value_counts()
#     print test_data[feat].value_counts()
train_data.columns

Index([u'service_type', u'is_mix_service', u'online_time', u'1_total_fee',
       u'2_total_fee', u'3_total_fee', u'4_total_fee', u'month_traffic',
       u'many_over_bill', u'contract_type', u'contract_time',
       u'is_promise_low_consume', u'net_service', u'pay_times', u'pay_num',
       u'last_month_traffic', u'local_trafffic_month', u'local_caller_time',
       u'service1_caller_time', u'service2_caller_time', u'gender', u'age',
       u'complaint_level', u'former_complaint_num', u'former_complaint_fee',
       u'current_service', u'user_id', u'stype_0', u'stype_1', u'ismix_0',
       u'ismix_1', u'mob_0', u'mob_1', u'ctype_0', u'ctype_1', u'ctype_2',
       u'ctype_3', u'ctype_4', u'ctype_5', u'ctype_6', u'ctype_7', u'ctype_8',
       u'islow_0', u'islow_1', u'ns_1', u'ns_2', u'ns_3', u'ns_4', u'gender_0',
       u'gender_1', u'gender_2', u'total_fee', u'1_2_total_fee', u'fee_month',
       u'call_time', u'traffic'],
      dtype='object')

In [8]:
feature = ['service_type', u'is_mix_service', u'online_time', u'1_total_fee',
       u'2_total_fee', u'3_total_fee', u'4_total_fee', u'month_traffic',
       u'many_over_bill', u'contract_type', u'contract_time',
       u'is_promise_low_consume', u'net_service', u'pay_times', u'pay_num',
       u'last_month_traffic', u'local_trafffic_month', u'local_caller_time',
       u'service1_caller_time', u'service2_caller_time', u'gender', u'age',
       u'complaint_level', u'former_complaint_num', u'former_complaint_fee',]

# 

train_x, evals_x, train_y, evals_y = train_test_split(train_data[feature], train_data.current_service,
                                                      test_size=0.2, random_state=2018)

print train_x.shape
print("LGB test")
clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=120, reg_alpha=0.0, reg_lambda=1,
    max_depth=-1, n_estimators=10000, objective='multi-class',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, min_child_weight=50, random_state=2018, n_jobs=-1,
    )
clf.fit(train_x, train_y, eval_set=[(train_x, train_y),(evals_x, evals_y)], 
        eval_metric='multi_error',early_stopping_rounds=500, 
        categorical_feature=['many_over_bill', 'gender', 'contract_type'],
        )

# 

(595190, 25)
LGB test


/usr/local/lib/python2.7/dist-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['contract_type', 'gender', 'many_over_bill']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	training's multi_error: 0.136698	valid_1's multi_error: 0.140136
Training until validation scores don't improve for 500 rounds.
[2]	training's multi_error: 0.117267	valid_1's multi_error: 0.11943
[3]	training's multi_error: 0.109819	valid_1's multi_error: 0.112313
[4]	training's multi_error: 0.105314	valid_1's multi_error: 0.108355
[5]	training's multi_error: 0.104558	valid_1's multi_error: 0.106702
[6]	training's multi_error: 0.101924	valid_1's multi_error: 0.104013
[7]	training's multi_error: 0.099748	valid_1's multi_error: 0.101776
[8]	training's multi_error: 0.0978679	valid_1's multi_error: 0.0999946
[9]	training's multi_error: 0.0965272	valid_1's multi_error: 0.0987849
[10]	training's multi_error: 0.0954922	valid_1's multi_error: 0.0976424
[11]	training's multi_error: 0.0946236	valid_1's multi_error: 0.0971182
[12]	training's multi_error: 0.0938053	valid_1's multi_error: 0.0960161
[13]	training's multi_error: 0.0931383	valid_1's multi_error: 0.0956464
[14]	training's multi_err

[115]	training's multi_error: 0.0672357	valid_1's multi_error: 0.0742954
[116]	training's multi_error: 0.0670643	valid_1's multi_error: 0.0742013
[117]	training's multi_error: 0.0669366	valid_1's multi_error: 0.0741609
[118]	training's multi_error: 0.0667602	valid_1's multi_error: 0.0741341
[119]	training's multi_error: 0.0666997	valid_1's multi_error: 0.0740534
[120]	training's multi_error: 0.0665519	valid_1's multi_error: 0.0739929
[121]	training's multi_error: 0.0663939	valid_1's multi_error: 0.0739324
[122]	training's multi_error: 0.0662461	valid_1's multi_error: 0.0738048
[123]	training's multi_error: 0.0661285	valid_1's multi_error: 0.0736569
[124]	training's multi_error: 0.0659604	valid_1's multi_error: 0.0736367
[125]	training's multi_error: 0.0658328	valid_1's multi_error: 0.0735628
[126]	training's multi_error: 0.0656815	valid_1's multi_error: 0.0734418
[127]	training's multi_error: 0.0654783	valid_1's multi_error: 0.073341
[128]	training's multi_error: 0.065364	valid_1's mul

[228]	training's multi_error: 0.0547523	valid_1's multi_error: 0.0682738
[229]	training's multi_error: 0.0546229	valid_1's multi_error: 0.0683343
[230]	training's multi_error: 0.0545708	valid_1's multi_error: 0.0682939
[231]	training's multi_error: 0.0544734	valid_1's multi_error: 0.0682805
[232]	training's multi_error: 0.0544381	valid_1's multi_error: 0.0683544
[233]	training's multi_error: 0.0542936	valid_1's multi_error: 0.068267
[234]	training's multi_error: 0.0542113	valid_1's multi_error: 0.0682805
[235]	training's multi_error: 0.0541373	valid_1's multi_error: 0.06822
[236]	training's multi_error: 0.0540584	valid_1's multi_error: 0.0681461
[237]	training's multi_error: 0.0539945	valid_1's multi_error: 0.0681394
[238]	training's multi_error: 0.0538887	valid_1's multi_error: 0.0680385
[239]	training's multi_error: 0.0538282	valid_1's multi_error: 0.0679915
[240]	training's multi_error: 0.0537307	valid_1's multi_error: 0.0679512
[241]	training's multi_error: 0.0536165	valid_1's mult

...

[2896]	training's multi_error: 0.0012349	valid_1's multi_error: 0.0551217
[2897]	training's multi_error: 0.00123154	valid_1's multi_error: 0.0551553
[2898]	training's multi_error: 0.00122986	valid_1's multi_error: 0.0551419
[2899]	training's multi_error: 0.0012265	valid_1's multi_error: 0.0551217
[2900]	training's multi_error: 0.00121978	valid_1's multi_error: 0.0551419
[2901]	training's multi_error: 0.00122146	valid_1's multi_error: 0.0551419
[2902]	training's multi_error: 0.00121306	valid_1's multi_error: 0.0551217
[2903]	training's multi_error: 0.0012181	valid_1's multi_error: 0.0551486
[2904]	training's multi_error: 0.00121978	valid_1's multi_error: 0.0551688
[2905]	training's multi_error: 0.0012181	valid_1's multi_error: 0.0551889
[2906]	training's multi_error: 0.0012181	valid_1's multi_error: 0.0551688
[2907]	training's multi_error: 0.00121978	valid_1's multi_error: 0.055162
[2908]	training's multi_error: 0.00122146	valid_1's multi_error: 0.0551419
[2909]	training's multi_error: 

[3006]	training's multi_error: 0.0010484	valid_1's multi_error: 0.0551486
[3007]	training's multi_error: 0.00104504	valid_1's multi_error: 0.0551688
[3008]	training's multi_error: 0.00105008	valid_1's multi_error: 0.0552024
[3009]	training's multi_error: 0.0010484	valid_1's multi_error: 0.0551755
[3010]	training's multi_error: 0.00104504	valid_1's multi_error: 0.0551351
[3011]	training's multi_error: 0.0010484	valid_1's multi_error: 0.055162
[3012]	training's multi_error: 0.00105008	valid_1's multi_error: 0.0552091
[3013]	training's multi_error: 0.00104672	valid_1's multi_error: 0.0552225
[3014]	training's multi_error: 0.00104168	valid_1's multi_error: 0.0552292
[3015]	training's multi_error: 0.00104168	valid_1's multi_error: 0.0552225
[3016]	training's multi_error: 0.00103832	valid_1's multi_error: 0.0552024
[3017]	training's multi_error: 0.00103832	valid_1's multi_error: 0.0552158
[3018]	training's multi_error: 0.00104	valid_1's multi_error: 0.0552091
[3019]	training's multi_error: 0

[3115]	training's multi_error: 0.000920714	valid_1's multi_error: 0.0552964
[3116]	training's multi_error: 0.000922395	valid_1's multi_error: 0.055283
[3117]	training's multi_error: 0.000922395	valid_1's multi_error: 0.0552696
[3118]	training's multi_error: 0.000915674	valid_1's multi_error: 0.055283
[3119]	training's multi_error: 0.000912314	valid_1's multi_error: 0.0553099
[3120]	training's multi_error: 0.000910634	valid_1's multi_error: 0.055283
[3121]	training's multi_error: 0.000919034	valid_1's multi_error: 0.0552897
[3122]	training's multi_error: 0.000910634	valid_1's multi_error: 0.0552964
[3123]	training's multi_error: 0.000907273	valid_1's multi_error: 0.0552091
[3124]	training's multi_error: 0.000910634	valid_1's multi_error: 0.0552763
[3125]	training's multi_error: 0.000908953	valid_1's multi_error: 0.0552763
[3126]	training's multi_error: 0.000915674	valid_1's multi_error: 0.0552628
[3127]	training's multi_error: 0.000910634	valid_1's multi_error: 0.0552628
[3128]	training

[3224]	training's multi_error: 0.000804785	valid_1's multi_error: 0.055283
[3225]	training's multi_error: 0.000801425	valid_1's multi_error: 0.0552964
[3226]	training's multi_error: 0.000798064	valid_1's multi_error: 0.0552628
[3227]	training's multi_error: 0.000799745	valid_1's multi_error: 0.055283
[3228]	training's multi_error: 0.000801425	valid_1's multi_error: 0.0552897
[3229]	training's multi_error: 0.000811506	valid_1's multi_error: 0.0552897
[3230]	training's multi_error: 0.000808145	valid_1's multi_error: 0.0553032
[3231]	training's multi_error: 0.000801425	valid_1's multi_error: 0.055283
[3232]	training's multi_error: 0.000806465	valid_1's multi_error: 0.0553166
[3233]	training's multi_error: 0.000804785	valid_1's multi_error: 0.0553099
[3234]	training's multi_error: 0.000808145	valid_1's multi_error: 0.0553233
[3235]	training's multi_error: 0.000803105	valid_1's multi_error: 0.0553569
[3236]	training's multi_error: 0.000801425	valid_1's multi_error: 0.0553032
[3237]	training

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        learning_rate=0.05, max_depth=-1, min_child_samples=20,
        min_child_weight=50, min_split_gain=0.0, n_estimators=10000,
        n_jobs=-1, num_leaves=120, objective='multi-class',
        random_state=2018, reg_alpha=0.0, reg_lambda=1, silent=True,
        subsample=0.7, subsample_for_bin=200000, subsample_freq=1)

In [24]:
feature = [u'service_type', u'is_mix_service', u'online_time', u'1_total_fee',
       u'2_total_fee', u'3_total_fee', u'4_total_fee', u'month_traffic',
       u'many_over_bill', u'contract_type', u'contract_time',
       u'is_promise_low_consume', u'net_service', u'pay_times', u'pay_num',
       u'last_month_traffic', u'local_trafffic_month', u'local_caller_time',
       u'service1_caller_time', u'service2_caller_time', u'gender', u'age',
       u'complaint_level', u'former_complaint_num', u'former_complaint_fee',
       u'stype_0', u'stype_1', u'ismix_0',
       u'ismix_1', u'mob_0', u'mob_1', u'ctype_0', u'ctype_1', u'ctype_2',
       u'ctype_3', u'ctype_4', u'ctype_5', u'ctype_6', u'ctype_7', u'ctype_8',
       u'islow_0', u'islow_1', u'ns_1', u'ns_2', u'ns_3', u'ns_4', u'gender_0',
       u'gender_1', u'gender_2', u'total_fee', u'1_2_total_fee', u'fee_month',
       u'call_time', u'traffic']

# 

train_x, evals_x, train_y, evals_y = train_test_split(train_data[feature], train_data.current_service,
                                                      test_size=0.2, random_state=2018)

print train_x.shape
print("LGB test")
clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=120, reg_alpha=0.0, reg_lambda=1,
    max_depth=-1, n_estimators=20000, objective='multi-class',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.1, min_child_weight=50, random_state=2018, n_jobs=-1,
    )
clf.fit(train_x, train_y, eval_set=[(train_x, train_y),(evals_x, evals_y)], 
        eval_metric='multi_error',early_stopping_rounds=500, 
        categorical_feature=['service_type', ],
        )

# 

(595190, 54)
LGB test


/usr/local/lib/python2.7/dist-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['service_type']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	training's multi_error: 0.127885	valid_1's multi_error: 0.130405
Training until validation scores don't improve for 500 rounds.
[2]	training's multi_error: 0.108327	valid_1's multi_error: 0.110129
[3]	training's multi_error: 0.103355	valid_1's multi_error: 0.105472
[4]	training's multi_error: 0.0984677	valid_1's multi_error: 0.100391
[5]	training's multi_error: 0.0953292	valid_1's multi_error: 0.0974072
[6]	training's multi_error: 0.0932845	valid_1's multi_error: 0.0959892
[7]	training's multi_error: 0.0924007	valid_1's multi_error: 0.0954179
[8]	training's multi_error: 0.0914733	valid_1's multi_error: 0.094672
[9]	training's multi_error: 0.0902098	valid_1's multi_error: 0.0935564
[10]	training's multi_error: 0.0895932	valid_1's multi_error: 0.0927701
[11]	training's multi_error: 0.0886524	valid_1's multi_error: 0.0921115
[12]	training's multi_error: 0.0878274	valid_1's multi_error: 0.0908749
[13]	training's multi_error: 0.0870394	valid_1's multi_error: 0.0903977
[14]	training's mu

[115]	training's multi_error: 0.0550849	valid_1's multi_error: 0.0692818
[116]	training's multi_error: 0.0548968	valid_1's multi_error: 0.0692079
[117]	training's multi_error: 0.0547976	valid_1's multi_error: 0.0690063
[118]	training's multi_error: 0.0546061	valid_1's multi_error: 0.0689257
[119]	training's multi_error: 0.05447	valid_1's multi_error: 0.0688585
[120]	training's multi_error: 0.0542365	valid_1's multi_error: 0.0687778
[121]	training's multi_error: 0.05405	valid_1's multi_error: 0.0686904
[122]	training's multi_error: 0.0538668	valid_1's multi_error: 0.0686568
[123]	training's multi_error: 0.0536215	valid_1's multi_error: 0.0685493
[124]	training's multi_error: 0.0534434	valid_1's multi_error: 0.0685426
[125]	training's multi_error: 0.0532855	valid_1's multi_error: 0.0684351
[126]	training's multi_error: 0.0532015	valid_1's multi_error: 0.0684216
[127]	training's multi_error: 0.0529579	valid_1's multi_error: 0.0682402
[128]	training's multi_error: 0.0527831	valid_1's multi

[228]	training's multi_error: 0.0389976	valid_1's multi_error: 0.0639121
[229]	training's multi_error: 0.0388363	valid_1's multi_error: 0.0637979
[230]	training's multi_error: 0.0387759	valid_1's multi_error: 0.0637576
[231]	training's multi_error: 0.0386162	valid_1's multi_error: 0.0638248
[232]	training's multi_error: 0.038497	valid_1's multi_error: 0.0637912
[233]	training's multi_error: 0.0383525	valid_1's multi_error: 0.0638315
[234]	training's multi_error: 0.0382802	valid_1's multi_error: 0.063892
[235]	training's multi_error: 0.0381256	valid_1's multi_error: 0.0637307
[236]	training's multi_error: 0.0380803	valid_1's multi_error: 0.0638181
[237]	training's multi_error: 0.0379307	valid_1's multi_error: 0.0637576
[238]	training's multi_error: 0.0378619	valid_1's multi_error: 0.0637038
[239]	training's multi_error: 0.0377661	valid_1's multi_error: 0.0636702
[240]	training's multi_error: 0.0376922	valid_1's multi_error: 0.0636568
[241]	training's multi_error: 0.037593	valid_1's mult

...

[1782]	training's multi_error: 0.000460357	valid_1's multi_error: 0.0559752
[1783]	training's multi_error: 0.000456997	valid_1's multi_error: 0.0560088
[1784]	training's multi_error: 0.000455317	valid_1's multi_error: 0.0559752
[1785]	training's multi_error: 0.000458677	valid_1's multi_error: 0.0559416
[1786]	training's multi_error: 0.000453637	valid_1's multi_error: 0.0559618
[1787]	training's multi_error: 0.000453637	valid_1's multi_error: 0.0559416
[1788]	training's multi_error: 0.000448596	valid_1's multi_error: 0.0559416
[1789]	training's multi_error: 0.000450276	valid_1's multi_error: 0.0558811
[1790]	training's multi_error: 0.000453637	valid_1's multi_error: 0.0558475
[1791]	training's multi_error: 0.000450276	valid_1's multi_error: 0.0558542
[1792]	training's multi_error: 0.000443556	valid_1's multi_error: 0.055861
[1793]	training's multi_error: 0.000441876	valid_1's multi_error: 0.0558677
[1794]	training's multi_error: 0.000445236	valid_1's multi_error: 0.055908
[1795]	trainin

[1890]	training's multi_error: 0.000379711	valid_1's multi_error: 0.0560357
[1891]	training's multi_error: 0.000379711	valid_1's multi_error: 0.0560223
[1892]	training's multi_error: 0.00037299	valid_1's multi_error: 0.0559887
[1893]	training's multi_error: 0.00037299	valid_1's multi_error: 0.0560357
[1894]	training's multi_error: 0.00037299	valid_1's multi_error: 0.0560626
[1895]	training's multi_error: 0.00037299	valid_1's multi_error: 0.0560424
[1896]	training's multi_error: 0.00036963	valid_1's multi_error: 0.0560155
[1897]	training's multi_error: 0.00036795	valid_1's multi_error: 0.056029
[1898]	training's multi_error: 0.00037635	valid_1's multi_error: 0.0560223
[1899]	training's multi_error: 0.000378031	valid_1's multi_error: 0.0560021
[1900]	training's multi_error: 0.00037299	valid_1's multi_error: 0.0560357
[1901]	training's multi_error: 0.00037131	valid_1's multi_error: 0.0560155
[1902]	training's multi_error: 0.00037635	valid_1's multi_error: 0.0560088
[1903]	training's multi

[1999]	training's multi_error: 0.000309145	valid_1's multi_error: 0.056029
[2000]	training's multi_error: 0.000307465	valid_1's multi_error: 0.0559752
[2001]	training's multi_error: 0.000302424	valid_1's multi_error: 0.0559819
[2002]	training's multi_error: 0.000304105	valid_1's multi_error: 0.0559416
[2003]	training's multi_error: 0.000304105	valid_1's multi_error: 0.0559887
[2004]	training's multi_error: 0.000305785	valid_1's multi_error: 0.0559954
[2005]	training's multi_error: 0.000304105	valid_1's multi_error: 0.0559349
[2006]	training's multi_error: 0.000307465	valid_1's multi_error: 0.0559685
[2007]	training's multi_error: 0.000302424	valid_1's multi_error: 0.0559551
[2008]	training's multi_error: 0.000302424	valid_1's multi_error: 0.0559349
[2009]	training's multi_error: 0.000305785	valid_1's multi_error: 0.0559416
[2010]	training's multi_error: 0.000299064	valid_1's multi_error: 0.0558811
[2011]	training's multi_error: 0.000294024	valid_1's multi_error: 0.0559887
[2012]	traini

[2107]	training's multi_error: 0.000262101	valid_1's multi_error: 0.0561701
[2108]	training's multi_error: 0.000262101	valid_1's multi_error: 0.056197
[2109]	training's multi_error: 0.000262101	valid_1's multi_error: 0.056197
[2110]	training's multi_error: 0.000262101	valid_1's multi_error: 0.0562306
[2111]	training's multi_error: 0.000263781	valid_1's multi_error: 0.0562037
[2112]	training's multi_error: 0.000262101	valid_1's multi_error: 0.0562104
[2113]	training's multi_error: 0.000260421	valid_1's multi_error: 0.0561903
[2114]	training's multi_error: 0.000257061	valid_1's multi_error: 0.0561567
[2115]	training's multi_error: 0.000262101	valid_1's multi_error: 0.0561634
[2116]	training's multi_error: 0.000262101	valid_1's multi_error: 0.0561836
[2117]	training's multi_error: 0.000262101	valid_1's multi_error: 0.0561365
[2118]	training's multi_error: 0.000258741	valid_1's multi_error: 0.0561096
[2119]	training's multi_error: 0.000265461	valid_1's multi_error: 0.0561567
[2120]	trainin

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=50, min_split_gain=0.0, n_estimators=20000,
        n_jobs=-1, num_leaves=120, objective='multi-class',
        random_state=2018, reg_alpha=0.0, reg_lambda=1, silent=True,
        subsample=0.7, subsample_for_bin=200000, subsample_freq=1)

In [19]:
test_data['predict'] = clf.predict(test_data[feature])
res = test_data.loc[:,['user_id','predict']]
res.describe()

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,predict
count,200000.000000
mean,1.831610
std,2.662786
min,0.000000
25%,0.000000
50%,0.000000
75%,4.000000
max,10.000000


In [20]:
res.predict = res.predict.replace(0, 90063345)
res.predict = res.predict.replace(1, 90109916)
res.predict = res.predict.replace(2, 90155946)
res.predict = res.predict.replace(3, 89950166)
res.predict = res.predict.replace(4, 89950167)
res.predict = res.predict.replace(5, 99999828)
res.predict = res.predict.replace(6, 89950168)
res.predict = res.predict.replace(7, 99999827)
res.predict = res.predict.replace(8, 99999826)
res.predict = res.predict.replace(9, 99999830)
res.predict = res.predict.replace(10, 99999825)
res

,user_id,predict
0,012pSke7DsCrR985,90063345
1,012re3ZJSB6FptHW,90063345
2,013ElTBrRntuZDXz,90063345
3,0146B9qQbNxIynoX,90063345
4,0149vWf6u8UrKyJT,90063345
5,014fpazsyUghObB3,90063345
6,014seANIFwmJRBo5,99999827
7,014zAhyR9Qr7ILOk,89950167
8,015R7bnaYKIwkuml,89950166
9,015sZpdVXKiB96a3,90063345


In [21]:
res.predict.value_counts()

90063345    116014
89950167     20571
89950166     17024
90109916      9894
99999828      8659
90155946      6910
99999826      5345
99999827      4358
99999830      4167
89950168      3848
99999825      3210
Name: predict, dtype: int64

In [23]:
res.predict.value_counts()

90063345    116014
89950167     20636
89950166     16927
90109916      9894
99999828      8674
90155946      6912
99999826      5259
99999827      4415
99999830      4209
89950168      3843
99999825      3217
Name: predict, dtype: int64

In [22]:
res.to_csv('sub/sub.csv',index = False)

In [21]:
def features(data):
    return data

def onehot(data):
    feats = ['service_type', 'is_mix_service', 'many_over_bill', 'contract_type', 'is_promise_low_consume',
             'net_service', 'gender']
    for feat in feats:
        data.feat
    
print train_data.net_service.value_counts()
print test_data.net_service.value_counts()

4    705857
3     25282
2     12504
9       347
Name: net_service, dtype: int64
4    194425
3      3723
2      1763
9        89
Name: net_service, dtype: int64


In [7]:
print train_data.loc[train_data.gender == -1]

        service_type  is_mix_service  online_time  1_total_fee  2_total_fee  \
140569             0               0           14         56.0        56.26   
232620             0               0           10         76.0        76.00   

        3_total_fee  4_total_fee  month_traffic  many_over_bill  \
140569         56.0         61.0       0.000000               0   
232620         76.0         76.0       3.122854               0   

        contract_type        ...         local_caller_time  \
140569              1        ...                  0.000000   
232620              1        ...                127.916667   

        service1_caller_time  service2_caller_time  gender  age  \
140569                   0.0              0.000000      -1   -1   
232620                   0.0            127.916667      -1   -1   

        complaint_level  former_complaint_num  former_complaint_fee  \
140569                0                     0                   0.0   
232620                0      

In [44]:
test_data.loc[test_data.service_type == 1].predict.describe()

count    132818.000000
mean          0.178470
std           0.500612
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           2.000000
Name: predict, dtype: float64

In [46]:
test_data.loc[test_data.service_type == 0].predict.describe()

count    67182.000000
mean         5.105237
std          2.130603
min          3.000000
25%          3.000000
50%          4.000000
75%          7.000000
max         10.000000
Name: predict, dtype: float64

In [53]:
print clf.feature_importances_
print train_x.columns
zy = pd.DataFrame()
zy['f'] = train_x.columns
zy['v'] = clf.feature_importances_
zy

[  544  1114 86200 88159 82968 85735 86465 88919  5761 26557 16177  2805
   902 20398 66677 45565 81114 78370 28893 95901 10638 61597  2375  1528
  2682   149    54   748   398  2999  1178  1365  2315  2879  1373  1098
    94    99   119     0  1641   791     0    68   307   415   876  3183
  2331]
Index([u'service_type', u'is_mix_service', u'online_time', u'1_total_fee',
       u'2_total_fee', u'3_total_fee', u'4_total_fee', u'month_traffic',
       u'many_over_bill', u'contract_type', u'contract_time',
       u'is_promise_low_consume', u'net_service', u'pay_times', u'pay_num',
       u'last_month_traffic', u'local_trafffic_month', u'local_caller_time',
       u'service1_caller_time', u'service2_caller_time', u'gender', u'age',
       u'complaint_level', u'former_complaint_num', u'former_complaint_fee',
       u'stype_0', u'stype_1', u'ismix_0', u'ismix_1', u'mob_0', u'mob_1',
       u'ctype_0', u'ctype_1', u'ctype_2', u'ctype_3', u'ctype_4', u'ctype_5',
       u'ctype_6', u'ctype_7',

,f,v
0,service_type,544
1,is_mix_service,1114
2,online_time,86200
3,1_total_fee,88159
4,2_total_fee,82968
5,3_total_fee,85735
6,4_total_fee,86465
7,month_traffic,88919
8,many_over_bill,5761
9,contract_type,26557
